In [12]:
import time
from math import sqrt

In [13]:
class DisjointSetUnion:
    def __init__(self, size):
        self.parent_ = list(range(size))
        self.ranks_ = [0] * size
        self.count = size

    def find(self, node):
        if self.parent_[node] != node:
            self.parent_[node] = self.find(self.parent_[node])
        return self.parent_[node]

    def union_sets(self, first, second):
        f_root = self.find(first)
        s_root = self.find(second)
        if f_root == s_root:
            return False
        if self.ranks_[f_root] < self.ranks_[s_root]:
            self.parent_[f_root] = s_root
        elif self.ranks_[f_root] > self.ranks_[s_root]:
            self.parent_[s_root] = f_root
        else:
            self.parent_[s_root] = f_root
            self.ranks_[f_root] += 1
        return True

    def result(self):
        return list(map(self.find, self.parent_))


In [14]:
def distance_1(title1, title2, num_of_words):
    # simple vec distance
    id1 = 0
    id2 = 0
    vec = []
    while id1 < len(title1) and id2 < len(title2):
        x = y = 0
        word1 = title1[id1][0]
        word2 = title2[id2][0]
        if word1 <= word2:
            x = title1[id1][1]
            id1 += 1
        if word1 >= word2:
            y = title2[id2][1]
            id2 += 1
        vec.append(abs(x - y))
    while id1 < len(title1):
        vec.append(title1[id1][1])
        id1 += 1
    while id2 < len(title2):
        vec.append(title2[id2][1])
        id2 += 1
    return sqrt(sum(map(lambda x: x * x, vec)))


In [15]:
def distance_2(title1, title2, num_of_words):
    # manhattan vec distance
    id1 = 0
    id2 = 0
    vec = []
    while id1 < len(title1) and id2 < len(title2):
        x = y = 0
        word1 = title1[id1][0]
        word2 = title2[id2][0]
        if word1 <= word2:
            x = title1[id1][1]
            id1 += 1
        if word1 >= word2:
            y = title2[id2][1]
            id2 += 1
        vec.append(abs(x - y))
    while id1 < len(title1):
        vec.append(title1[id1][1])
        id1 += 1
    while id2 < len(title2):
        vec.append(title2[id2][1])
        id2 += 1
    return sum(vec)


In [16]:
def distance_3(title1, title2, num_of_words):
    # Pearson
    x = [0] * num_of_words
    for elem in title1:
        x[elem[0] - 1] += elem[1]
    y = [0] * num_of_words
    for elem in title2:
        y[elem[0] - 1] += elem[1]
    x_m = sum(x) / len(x)
    y_m = sum(y) / len(y)
    x = [x[_] - x_m for _ in range(len(x))]
    y = [y[_] - y_m for _ in range(len(x))]
    a = sum([x[_] * y[_] for _ in range(len(x))])
    b = sum([x[_] * x[_] for _ in range(len(x))])
    c = sum([y[_] * y[_] for _ in range(len(x))])
    if a == 0:
        return 1
    return 1 - (a / sqrt(b * c))


Здесь мы реализуем кластеризацию на основе минимального остовного дерева, максимизирующую минимальное межклассовое расстояние.
За основу взят алгоитм Крускала.

In [22]:
def cluster(ver, eds, num_of_clusters):
    # return DisjointSetUnion of edges
    res = DisjointSetUnion(len(ver))
    # Your code here
    eds.sort(key = lambda eds:eds[0])
    n = len(eds)
    for i in range(n):
        first, second = edges[i][1], edges[i][2]
        if res.union_sets(first, second):
            res.count -= 1
        if res.count == num_of_clusters:
            break

    return res

In [23]:
import numpy as np
from glob import glob

Попробуем разбиение с первой функцией дистанции

In [ ]:
files = []
files += glob('test_50.*')
files += glob('test_400.*')
files += glob('test_1000.*')
distances = [distance_1, distance_2, distance_3]
print(files)


for filename in files:
    print('--------------------------------------------------------------')
    print('Работа с файлом:', filename)
    
    for i in range(3):
        
        distance = distances[i]
        print('Рассматриваем', i + 1, 'функцию для определения дистанции')
    
        t_start = time.time()

        # clusters num
        clusters_num = 8

        # word freq param
        min_count = 2
        max_count = 10

        vertices = []
        with open(filename) as f:
            doc_num = int(f.readline())
            words_num = int(f.readline())
            lines_num = int(f.readline())
            words_count = [0] * words_num
            vertices = [[] for _ in range(doc_num)]
            for i in range(lines_num):
                docID, wordID, count = map(int, f.readline().split())
                words_count[wordID - 1] += count
                if count <= max_count:
                    vertices[docID - 1].append([wordID, count])

        new_v = []
        for elem in vertices:
            new_v.append([])
            for k in elem:
                if min_count <= words_count[k[0] - 1] <= max_count:
                    new_v[-1].append(k)
            if len(new_v[-1]) == 0:
                print("Warning: vertex without words")
        vertices = new_v
        print('Vertices:', len(vertices))

        edges = [(distance(vertices[i], vertices[j], words_num), i, j) for i in range(len(vertices) - 1) for j in range(i + 1, len(vertices))]
        print('Edges:', len(edges))

        components = cluster(vertices, edges, clusters_num)

        comp = components.result()
        d = dict()
        for i in range(len(comp)):
            if comp[i] not in d:
                d[comp[i]] = [i + 1]
            else:
                d[comp[i]].append(i + 1)
        for elem in d:
            print("Cluster", elem, ":", len(d[elem]), "elems", ":", d[elem])

        print("Time:", time.time() - t_start)


['test_50.txt', 'test_400.txt', 'test_1000.txt']
--------------------------------------------------------------
Работа с файлом: test_50.txt
Рассматриваем 1 функцию для определения дистанции
Vertices: 50
Edges: 1225
Cluster 3 : 42 elems : [1, 2, 3, 4, 5, 8, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 6 : 1 elems : [7]
Cluster 8 : 1 elems : [9]
Cluster 17 : 1 elems : [18]
Cluster 21 : 2 elems : [22, 24]
Cluster 39 : 1 elems : [40]
Cluster 47 : 1 elems : [48]
Time: 0.29383301734924316
Рассматриваем 2 функцию для определения дистанции
Vertices: 50
Edges: 1225
Cluster 3 : 42 elems : [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 45, 46, 47, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 6 : 1 elems : [7]
Cluster 19 : 1 elems : [20]
Cluster 21 : 2 elems : [22, 24]
Cluste